In [13]:
#필요 라이브러리 임포트

import os
import cv2
import json
import math  # math 모듈 임포트 추가
import shutil
import numpy as np
import matplotlib.pyplot as plt
from itertools import product
from sklearn.utils import class_weight
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc

In [14]:
# 이미지 리사이징 함수
def resize_image(image_path, target_size):
    """
    이미지를 지정한 크기로 리사이즈합니다.
    :param image_path: 원본 이미지 경로
    :param target_size: 리사이즈할 크기 (width, height)
    :return: 리사이즈된 이미지
    """
    image = cv2.imread(image_path)
    if image is None:
        print(f"Failed to load image: {image_path}")
        return None
    resized_image = cv2.resize(image, target_size, interpolation=cv2.INTER_AREA)
    return resized_image

# 디렉토리 내 모든 이미지를 리사이즈하여 저장
def makedir_resize(input_folder, output_folder, target_size):
    """
    폴더 내 이미지를 리사이즈하여 저장하는 함수
    :param input_folder: 원본 이미지가 저장된 폴더
    :param output_folder: 리사이즈된 이미지를 저장할 폴더
    :param target_size: 리사이즈할 크기 (width, height)
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    for image_name in os.listdir(input_folder):
        image_path = os.path.join(input_folder, image_name)
        if not image_name.lower().endswith((".jpg", ".png")):
            print(f"Skipped non-image file: {image_name}")
            continue

        resized_image = resize_image(image_path, target_size)
        if resized_image is not None:
            output_image_path = os.path.join(output_folder, image_name)
            cv2.imwrite(output_image_path, resized_image)
            print(f"Resized image saved to: {output_image_path}")





In [15]:
# 이미지 회전 함수
def rotate_image(image, angle):
    """
    이미지를 특정 각도로 회전시킵니다.
    :param image: 원본 이미지
    :param angle: 회전할 각도
    :return: 회전된 이미지
    """
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
    # 이미지의 새로운 경계 계산
    cos = np.abs(rotation_matrix[0, 0])
    sin = np.abs(rotation_matrix[0, 1])
    new_w = int((h * sin) + (w * cos))
    new_h = int((h * cos) + (w * sin))

    # 회전 행렬에 새로운 크기 적용 (중심 이동)
    rotation_matrix[0, 2] += (new_w / 2) - center[0]
    rotation_matrix[1, 2] += (new_h / 2) - center[1]

    # 회전 및 Zero Padding 적용
    rotated_image = cv2.warpAffine(
        image, 
        rotation_matrix, 
        (new_w, new_h), 
        borderValue=(0, 0, 0)  # Zero Padding
    )
    return rotated_image

# 데이터 증강 함수 (회전)
def makedir_rotate(input_folder, output_folder, n):
    """
    데이터셋을 n배로 증강. 각 이미지를 (360/n * k)도로 회전 (k=1, ... ,n-1)
    :param input_folder: 원본 이미지가 저장된 폴더
    :param output_folder: 회전된 이미지를 저장할 폴더
    :param n: 회전 각도 분할 수
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    angles = [(360 / n) * i for i in range(1, n)]
    
    for filename in os.listdir(input_folder):
        if not filename.lower().endswith((".jpg", ".png")):
            print(f"Skipped non-image file: {filename}")
            continue

        image_path = os.path.join(input_folder, filename)
        image = cv2.imread(image_path)
        if image is None:
            print(f"Failed to load image: {image_path}")
            continue
        
        for angle in angles:
            rotated_image = rotate_image(image, angle)
            
            # 새로운 파일명 생성
            name, ext = os.path.splitext(filename)
            rotated_filename = f"{name}_rotated_{int(angle)}{ext}"
            
            # 회전된 이미지 저장
            cv2.imwrite(os.path.join(output_folder, rotated_filename), rotated_image)
            print(f"Saved rotated image: {rotated_filename}")
        
        # 원본 이미지도 저장
        cv2.imwrite(os.path.join(output_folder, filename), image)
        print(f"Original image saved: {filename}")

In [16]:
def crop_zeropadding(image, size, position):
    """
    이미지를 4개의 모서리를 기준으로 지정된 크기로 자릅니다. 
    잘라낸 영역이 이미지 범위를 벗어날 경우 제로 패딩을 적용합니다.
    
    :param image: 원본 이미지 (numpy 배열)
    :param size: 자를 윈도우의 크기 (width, height) 튜플
    :param position: 자를 위치, 'top-left', 'top-right', 'bottom-left', 'bottom-right' 중 하나
    :return: 잘라낸 이미지 (지정된 크기와 동일한 numpy 배열)
    """
    window_w, window_h = size
    img_h, img_w = image.shape[:2]

    # 위치에 따른 시작 좌표 계산
    if position == 'top-left':
        x_start, y_start = 0, 0
    elif position == 'top-right':
        x_start, y_start = img_w - window_w, 0
    elif position == 'bottom-left':
        x_start, y_start = 0, img_h - window_h
    elif position == 'bottom-right':
        x_start, y_start = img_w - window_w, img_h - window_h
    else:
        raise ValueError("position 파라미터는 'top-left', 'top-right', 'bottom-left', 'bottom-right' 중 하나여야 합니다.")

    # 잘라낼 영역의 끝 좌표
    x_end = x_start + window_w
    y_end = y_start + window_h

    # 제로 패딩을 위한 빈 이미지 생성 (검은색)
    cropped = np.zeros((window_h, window_w, image.shape[2]), dtype=image.dtype)

    # 원본 이미지 내에서 자를 수 있는 영역 계산
    x1 = max(x_start, 0)
    y1 = max(y_start, 0)
    x2 = min(x_end, img_w)
    y2 = min(y_end, img_h)

    # 잘라낸 이미지를 배치할 위치 계산
    pad_x = max(-x_start, 0)
    pad_y = max(-y_start, 0)

    # 원본 이미지에서 잘라낸 부분을 새로운 이미지에 복사
    cropped[pad_y:pad_y + (y2 - y1), pad_x:pad_x + (x2 - x1)] = image[y1:y2, x1:x2]

    return cropped

def makedir_crop(input_folder, output_folder, crop_size=(200, 200)):
    """
    입력 폴더 내의 모든 이미지를 네 모서리에서 지정된 크기로 자르고, 
    잘라낸 이미지를 출력 폴더에 저장합니다.
    
    :param input_folder: 원본 이미지가 저장된 폴더
    :param output_folder: 잘라낸 이미지를 저장할 폴더
    :param crop_size: 잘라낼 윈도우의 크기 (width, height) 튜플. 기본값은 (200, 200)
    """
    # 출력 폴더가 존재하지 않으면 생성
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
        print(f"출력 폴더 생성: {output_folder}")

    # 잘라낼 위치 목록
    positions = ['top-left', 'top-right', 'bottom-left', 'bottom-right']

    # 입력 폴더 내의 모든 파일 처리
    for filename in os.listdir(input_folder):
        if not filename.lower().endswith((".jpg", ".jpeg", ".png", ".bmp", ".tiff")):
            print(f"이미지가 아닌 파일을 건너뜁니다: {filename}")
            continue

        image_path = os.path.join(input_folder, filename)
        image = cv2.imread(image_path)
        if image is None:
            print(f"이미지를 불러오는 데 실패했습니다: {image_path}")
            continue

        for position in positions:
            cropped_image = crop_zeropadding(image, crop_size, position)

            # 파일명 생성: 원본명 + 위치명 + 확장자
            name, ext = os.path.splitext(filename)
            cropped_filename = f"{name}_{position}{ext}"

            # 잘라낸 이미지 저장
            save_path = os.path.join(output_folder, cropped_filename)
            cv2.imwrite(save_path, cropped_image)
            print(f"저장된 잘라낸 이미지: {cropped_filename}")

    print("모든 이미지에 대한 크롭 작업이 완료되었습니다.")

In [17]:
import os
import shutil
from PIL import Image
import numpy as np

def add_noise(input_folder, output_folder, noise_factor=0.1):
    """
    원본 이미지에 가우지안 노이즈를 추가합니다.
    
    :param input_folder: 원본 이미지가 저장된 입력 폴더 경로 (문자열)
    :param output_folder: 노이즈가 추가된 이미지가 저장될 출력 폴더 경로 (문자열)
    :param noise_factor: 노이즈의 강도 (기본값: 0.1)
    """
    # 입력 폴더 존재 여부 확인
    if not os.path.exists(input_folder):
        raise FileNotFoundError(f"Input folder '{input_folder}' does not exist.")
    
    # 출력 폴더가 없으면 생성
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
        print(f"Created output folder '{output_folder}'.")
    
    # 입력 폴더 내 모든 파일 순회
    for root, dirs, files in os.walk(input_folder):
        # 현재 폴더의 상대 경로 계산
        rel_path = os.path.relpath(root, input_folder)
        # 출력 폴더 내 해당 상대 경로에 해당하는 디렉터리 경로 생성
        output_dir = os.path.join(output_folder, rel_path)
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        
        for file in files:
            # 이미지 파일 확장자 확인
            if file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff', '.gif')):
                input_path = os.path.join(root, file)
                try:
                    # 이미지 열기 및 RGB로 변환
                    image = Image.open(input_path).convert('RGB')
                    image_np = np.array(image)
                    
                    # 가우시안 노이즈 생성
                    noise = np.random.normal(0, noise_factor * 255, image_np.shape).astype(np.float32)
                    noisy_image = image_np + noise
                    noisy_image = np.clip(noisy_image, 0, 255).astype(np.uint8)
                    
                    # 노이즈 이미지 PIL 객체로 변환
                    noisy_image_pil = Image.fromarray(noisy_image)
                    
                    # 증강된 이미지 이름 생성 (예: originalname_noisy.jpg)
                    base, ext = os.path.splitext(file)
                    noisy_filename = f"{base}_noisy{ext}"
                    output_path = os.path.join(output_dir, noisy_filename)
                    
                    # 노이즈 이미지 저장
                    noisy_image_pil.save(output_path)
                    
                except Exception as e:
                    print(f"Failed to process image '{input_path}': {e}")
    
    print("Data augmentation with Gaussian noise completed.")


In [18]:
def convert_history_to_native(history):
    """
    Converts all numpy data types in the history dictionary to native Python types.
    
    :param history: Dictionary containing training history.
    :return: Converted dictionary with native Python types.
    """
    native_history = {}
    for key, values in history.items():
        native_history[key] = [float(v) for v in values]
    return native_history


In [19]:
def save_training_results(results_base_dir, model, history, batch_size, epochs, neurons, dropout):
    """
    하이퍼파라미터 정보를 포함한 디렉토리를 생성하고 그 안에 학습 결과를 저장합니다.
    :param results_base_dir: 결과를 저장할 기본 디렉토리
    :param model: 학습된 모델
    :param history: 모델 학습 히스토리 (history 객체)
    :param batch_size: 배치 크기
    :param epochs: 에포크 수
    :param neurons: 각 레이어의 뉴런 개수 (리스트)
    :param dropout: 드롭아웃 비율
    """
    # 하이퍼파라미터 기반 디렉토리 이름 생성
    dir_name = f"bs{batch_size}_ep{epochs}_neurons{'-'.join(map(str, neurons))}_dr{dropout}"
    results_dir = os.path.join(results_base_dir, dir_name)

    # 디렉토리 생성
    if not os.path.exists(results_dir):
        os.makedirs(results_dir)

    # 모델 저장
    model_save_path = os.path.join(results_dir, 'model.h5')
    model.save(model_save_path)
    print(f"Model saved at: {model_save_path}")

    # 손실 그래프 저장
    plt.figure()
    plt.plot(history.history['loss'], label='Train Loss', color='blue')
    plt.plot(history.history['val_loss'], label='Validation Loss', color='orange')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    loss_graph_path = os.path.join(results_dir, 'loss_graph.png')
    plt.savefig(loss_graph_path, bbox_inches='tight')
    print(f"Loss graph saved at: {loss_graph_path}")
    plt.close()

    # 정확도 그래프 저장
    plt.figure()
    plt.plot(history.history['accuracy'], label='Train Accuracy', color='blue')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy', color='orange')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True)
    accuracy_graph_path = os.path.join(results_dir, 'accuracy_graph.png')
    plt.savefig(accuracy_graph_path, bbox_inches='tight')
    print(f"Accuracy graph saved at: {accuracy_graph_path}")
    plt.close()

    # 학습 기록 저장 (JSON)
    history_path = os.path.join(results_dir, 'training_history.json')
    native_history = convert_history_to_native(history.history)
    with open(history_path, 'w') as f:
        json.dump(native_history, f, ensure_ascii=False, indent=4)
    print(f"Training history saved at: {history_path}")

    # 하이퍼파라미터 정보 저장 (JSON)
    hyperparams_path = os.path.join(results_dir, 'hyperparameters.json')
    hyperparameters = {
        'batch_size': batch_size,
        'epochs': epochs,
        'neurons': neurons,
        'dropout': dropout
    }
    with open(hyperparams_path, 'w') as f:
        json.dump(hyperparameters, f, ensure_ascii=False, indent=4)
    print(f"Hyperparameters saved at: {hyperparams_path}")


In [26]:
def build_model(IMAGE_SIZE, NEURONS, DROPOUT):
    """
    주어진 뉴런 수와 드롭아웃 비율을 사용하여 모델을 구축합니다.
    
    :param IMAGE_SIZE: 입력 이미지의 크기 (예: (224, 224))
    :param NEURONS: 각 Conv2D 및 Dense 레이어의 뉴런 개수 (리스트)
                    예: [32, 64, 128, 256] (마지막 요소는 Dense 레이어)
    :param DROPOUT: 드롭아웃 비율 (float)
    :return: 컴파일된 Keras 모델
    """
    model = models.Sequential([
        layers.Input(shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3)),
        
        layers.Conv2D(NEURONS[0], (3, 3)),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.MaxPooling2D(2, 2),

        layers.Conv2D(NEURONS[1], (3, 3)),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.MaxPooling2D(2, 2),

        layers.Conv2D(NEURONS[2], (3, 3)),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.MaxPooling2D(2, 2),
    
        layers.Flatten(),
        layers.Dense(NEURONS[3]),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.Dropout(DROPOUT),
        layers.Dense(1, activation='sigmoid')  # 이진 분류를 위한 시그모이드 활성화 함수
    ])
    
    # 모델 컴파일
    model.compile(optimizer=Adam(),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    
    return model


def training_model(results_base_dir, build_model_fn, train_dir, IMAGE_SIZE, BATCH_SIZE, EPOCHS, DROPOUT, NEURONS):
    """
    주어진 하이퍼파라미터 조합으로 모델을 학습시키고 결과를 저장합니다.
    
    :param results_base_dir: 학습 결과를 저장할 기본 디렉터리
    :param build_model_fn: 모델을 구축하는 함수
    :param train_dir: 훈련 데이터가 저장된 디렉터리 경로
    :param IMAGE_SIZE: 입력 이미지의 크기 (예: (224, 224))
    :param BATCH_SIZE: 배치 사이즈 리스트
    :param EPOCHS: 에포크 수 리스트
    :param DROPOUT: 드롭아웃 비율 리스트
    :param NEURONS: 각 레이어의 뉴런 개수 리스트 (리스트의 리스트)
                    예: [[32, 64, 128, 256], [64, 128, 256, 512]]
    """
    # 모든 하이퍼파라미터 조합 생성
    hyperparameter_combinations = list(product(BATCH_SIZE, EPOCHS, DROPOUT, NEURONS))

    print(f"총 {len(hyperparameter_combinations)}개의 하이퍼파라미터 조합을 학습합니다.")

    for idx, (batch_size, epochs, dropout, neurons) in enumerate(hyperparameter_combinations, 1):
        print(f"\n학습 시작 {idx}/{len(hyperparameter_combinations)}: "
              f"Batch Size={batch_size}, Epochs={epochs}, Dropout={dropout}, Neurons={neurons}")

        # 데이터 제너레이터 설정
        train_datagen = ImageDataGenerator(
            rescale=1./255,
            validation_split=0.1,  # 훈련 데이터의 10%를 검증 데이터로 사용
            #최고 성능 모델에서는 0.2였음.
            width_shift_range=0.1,
            height_shift_range=0.1,
            shear_range=0.1,
            zoom_range=0.1,
            horizontal_flip=True,
            fill_mode='nearest'
            
        )
        
        
        seed=42
        # 훈련 데이터 제너레이터
        train_generator = train_datagen.flow_from_directory(
            train_dir,
            target_size=IMAGE_SIZE,
            batch_size=batch_size,
            class_mode='binary',  # 이진 분류를 위한 설정
            subset='training',
            shuffle=True,
            seed = seed
        )

        # 검증 데이터 제너레이터
        validation_generator = train_datagen.flow_from_directory(
            train_dir,
            target_size=IMAGE_SIZE,
            batch_size=batch_size,
            class_mode='binary',  # 이진 분류를 위한 설정
            subset='validation',
            shuffle=False,
            seed = seed
        )

        # 클래스 가중치 계산 (훈련 데이터 기반)
        classes = train_generator.classes
        class_weights_vals = class_weight.compute_class_weight('balanced', classes=np.unique(classes), y=classes)
        class_weights_dict = dict(enumerate(class_weights_vals))

        # 모델 구축
        model = build_model_fn(IMAGE_SIZE, neurons, dropout)

        rlr = ReduceLROnPlateau(
            monitor='val_loss',      # 모니터링할 지표
            factor=0.2,              # 학습률 감소 비율
            patience=5,              # 개선이 없을 때 기다릴 에포크 수
            min_lr=1e-7,             # 학습률의 최소값
            verbose=1                # 로그 출력
        )
        
        # 모델 학습
        history = model.fit(
            train_generator,
            steps_per_epoch=math.ceil(train_generator.samples / batch_size),
            epochs=epochs,
            validation_data=validation_generator,
            validation_steps=math.ceil(validation_generator.samples / batch_size),
            verbose=1,  # 학습 과정을 출력
            class_weight=class_weights_dict,
            callbacks=[rlr]  
        )

        # 학습 결과 저장
        save_training_results(
            results_base_dir=results_base_dir,
            model=model,
            history=history,
            batch_size=batch_size,
            epochs=epochs,
            neurons=neurons,
            dropout=dropout
        )

        print(f"학습 완료 {idx}/{len(hyperparameter_combinations)}")

    print("\n모든 하이퍼파라미터 조합에 대한 학습이 완료되었습니다.")


In [21]:
def sortmodel(models_dir, option=0):
    """
    주어진 디렉터리 내의 모델들을 정확도 또는 손실을 기준으로 정렬합니다.

    :param models_dir: 정렬할 모델들이 저장되어 있는 디렉터리 경로 (문자열)
    :param option: 정렬 기준 (0=정확도, 1=손실), 기본값은 0
    """
    
    # 모델 디렉터리 내의 모든 하위 디렉터리 가져오기
    model_dirs = [os.path.join(models_dir, d) for d in os.listdir(models_dir)
                  if os.path.isdir(os.path.join(models_dir, d))]

    model_performance = []

    for model_dir in model_dirs:
        # 학습 기록 파일 경로
        history_path = os.path.join(model_dir, 'training_history.json')

        if os.path.exists(history_path):
            with open(history_path, 'r') as f:
                history = json.load(f)
            
            # 마지막 에포크의 정확도와 손실 가져오기
            final_accuracy = history['val_accuracy'][-1]
            final_loss = history['val_loss'][-1]

            # 모델 정보 저장
            model_info = {
                'model_dir': model_dir,
                'accuracy': final_accuracy,
                'loss': final_loss
            }
            model_performance.append(model_info)
        else:
            print(f"Warning: '{history_path}' 파일이 존재하지 않습니다.")

    # 정렬 기준 설정
    if option == 0:
        # 정확도를 기준으로 내림차순 정렬
        sorted_models = sorted(model_performance, key=lambda x: x['accuracy'], reverse=True)
    elif option == 1:
        # 손실을 기준으로 오름차순 정렬
        sorted_models = sorted(model_performance, key=lambda x: x['loss'])
    else:
        print("Invalid option. Please choose 0 for accuracy or 1 for loss.")
        return

    # 정렬된 모델 정보 출력
    print(f"{'Rank':<5}{'Model Directory':<50}{'Accuracy':<10}{'Loss':<10}")
    for idx, model in enumerate(sorted_models, 1):
        print(f"{idx:<5}{model['model_dir']:<50}{model['accuracy']:<10.4f}{model['loss']:<10.4f}")

    return sorted_models


def show_good_model(sorted_models, n=5):
    """
    상위 n개의 모델 정보를 출력하고, 정확도와 손실을 시각화합니다.
    
    :param sorted_models: 특정 기준으로 정렬된 모델들 (리스트 of dicts)
    :param n: 정보를 몇 번째까지 출력할지 나타내는 정수 (기본값=5)
    """
    # 상위 n개의 모델 선택
    top_models = sorted_models[:n]
    
    if not top_models:
        print("정렬된 모델 리스트가 비어있습니다.")
        return
    
    # 상위 n개의 모델 정보 출력
    print(f"{'Rank':<5}{'Model Directory':<60}{'Accuracy':<10}{'Loss':<10}")
    for idx, model_info in enumerate(top_models, 1):
        model_dir = model_info['model_dir']
        accuracy = model_info['accuracy']
        loss = model_info['loss']
        print(f"{idx:<5}{model_dir:<60}{accuracy:<10.4f}{loss:<10.4f}")
    
    # 정확도 및 손실 그래프 시각화
    accuracies = [model['accuracy'] for model in top_models]
    losses = [model['loss'] for model in top_models]
    model_names = [os.path.basename(model['model_dir']) for model in top_models]
    
    # 그래프 크기 설정
    plt.figure(figsize=(12, 6))
    
    # 정확도 막대 그래프
    plt.subplot(1, 2, 1)
    bars = plt.bar(model_names, accuracies, color='skyblue')
    plt.xlabel('Model')
    plt.ylabel('Validation Accuracy')
    plt.title('Top Models Validation Accuracy')
    plt.ylim(0, 1)
    plt.xticks(rotation=45, ha='right')
    for bar, acc in zip(bars, accuracies):
        plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height() + 0.01, f"{acc:.4f}", ha='center', va='bottom')
    
    # 손실 막대 그래프
    plt.subplot(1, 2, 2)
    bars = plt.bar(model_names, losses, color='salmon')
    plt.xlabel('Model')
    plt.ylabel('Validation Loss')
    plt.title('Top Models Validation Loss')
    plt.xticks(rotation=45, ha='right')
    for bar, loss in zip(bars, losses):
        plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height() + 0.01, f"{loss:.4f}", ha='center', va='bottom')
    
    plt.tight_layout()
    plt.show()
    
    # 선택 사항: 각 모델의 구조 요약 출력
    print("\n=== 상위 모델들의 구조 요약 ===")
    for idx, model_info in enumerate(top_models, 1):
        model_path = os.path.join(model_info['model_dir'], 'model.h5')
        if os.path.exists(model_path):
            print(f"\nRank {idx}: {model_info['model_dir']}")
            model = load_model(model_path)
            model.summary()
        else:
            print(f"\nRank {idx}: 모델 파일이 존재하지 않습니다: {model_path}")
    

In [22]:

def test_evaluate(test_dir, model_dir, save_dir, IMAGE_SIZE=(200, 200), BATCH_SIZE=32):
    """
    학습된 모델을 테스트 데이터에 대해 평가하고, 결과를 저장합니다.
    
    :param test_dir: 테스트 데이터가 있는 디렉터리 경로
    :param model_dir: 평가할 모델의 디렉터리 경로 (model.h5 파일이 위치)
    :param save_dir: 평가 결과를 저장할 디렉터리 경로
    :param IMAGE_SIZE: 입력 이미지의 크기 (기본값=(200, 200))
    :param BATCH_SIZE: 배치 사이즈 (기본값=32)
    """
    # 필요한 디렉터리 생성
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
        print(f"저장 디렉터리 생성: {save_dir}")
    
    # 모델 파일 경로
    model_path = os.path.join(model_dir, 'model.h5')
    
    if not os.path.exists(model_path):
        print(f"Error: 모델 파일이 존재하지 않습니다: {model_path}")
        return
    
    # 모델 로드
    model = load_model(model_path)
    print(f"모델 로드 완료: {model_path}")
    
    # 테스트 데이터 제너레이터 설정
    test_datagen = ImageDataGenerator(rescale=1./255)
    
    test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary',  # 이진 분류인 경우 'binary', 다중 클래스인 경우 'categorical'
        shuffle=False  # 평가 시 데이터 순서를 유지
    )
    
    # 클래스 인덱스 확인
    print('Class indices:', test_generator.class_indices)
    
    # 모델 평가
    loss, accuracy = model.evaluate(test_generator, verbose=1)
    print(f"테스트 손실: {loss}")
    print(f"테스트 정확도: {accuracy}")
    
    # 예측 생성
    Y_pred = model.predict(test_generator)
    y_pred = (Y_pred > 0.5).astype(int).reshape(-1)  # 이진 분류인 경우
    
    
    # 실제 클래스
    y_true = test_generator.classes
    
    # 분류 보고서 생성
    report = classification_report(y_true, y_pred, target_names=list(test_generator.class_indices.keys()))
    print("분류 보고서:")
    print(report)
    
    # 분류 보고서를 JSON으로 저장
    report_dict = classification_report(y_true, y_pred, target_names=list(test_generator.class_indices.keys()), output_dict=True)
    report_path = os.path.join(save_dir, 'classification_report.json')
    with open(report_path, 'w') as f:
        json.dump(report_dict, f, indent=4)
    print(f"분류 보고서 JSON 저장: {report_path}")
    
    # 혼동 행렬 생성
    cm = confusion_matrix(y_true, y_pred)
    print("혼동 행렬:")
    print(cm)
    
    # 혼동 행렬 시각화 및 저장
    plt.figure(figsize=(6,6))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title('Confusion Matrix')
    plt.colorbar()
    classes = list(test_generator.class_indices.keys())
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    
    # 값 표시
    fmt = 'd'
    thresh = cm.max() / 2.
    for i, j in np.ndindex(cm.shape):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
    
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()
    cm_path = os.path.join(save_dir, 'confusion_matrix.png')
    plt.savefig(cm_path)
    plt.close()
    print(f"혼동 행렬 이미지 저장: {cm_path}")
    
    # ROC 곡선 및 AUC 계산 (이진 분류인 경우)
    if len(test_generator.class_indices) == 2:
        fpr, tpr, thresholds = roc_curve(y_true, Y_pred)
        roc_auc = auc(fpr, tpr)
        
        plt.figure()
        plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
        plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Receiver Operating Characteristic (ROC)')
        plt.legend(loc="lower right")
        roc_path = os.path.join(save_dir, 'roc_curve.png')
        plt.savefig(roc_path)
        plt.close()
        print(f"ROC 곡선 이미지 저장: {roc_path}")
    
    # 결과 요약 저장
    summary = {
        'test_loss': loss,
        'test_accuracy': accuracy,
        'classification_report': report_dict,
        'confusion_matrix': cm.tolist()  # numpy 배열을 리스트로 변환
    }
    
    if test_generator.num_classes == 1:
        summary['roc_auc'] = roc_auc
    
    summary_path = os.path.join(save_dir, 'evaluation_summary.json')
    with open(summary_path, 'w') as f:
        json.dump(summary, f, indent=4)
    print(f"평가 요약 저장: {summary_path}")
    
    # 최고 모델 디렉터리의 그래프 파일 복사
    graph_files = ['loss_graph.png', 'accuracy_graph.png']  # 복사할 그래프 파일 목록
    for graph_file in graph_files:
        source_path = os.path.join(model_dir, graph_file)
        if os.path.exists(source_path):
            destination_path = os.path.join(save_dir, f"best_{graph_file}")
            shutil.copy(source_path, destination_path)
            print(f"{graph_file} 복사하여 저장: {destination_path}")
        else:
            print(f"Warning: '{source_path}' 파일이 존재하지 않습니다. 복사하지 않습니다.")
    

    
    print("모델 평가가 완료되었습니다.")

In [23]:
import os
import json
import matplotlib.pyplot as plt

def train_history(model_dir):
    """
    주어진 모델 디렉터리에서 학습 내역을 로드하고, 손실 및 정확도 그래프를 시각화하며,
    주요 학습 요약 정보를 출력합니다.
    
    :param model_dir: 학습 내역을 보고자 하는 모델의 디렉터리 경로 (문자열)
    """
    # 필요한 파일 경로 설정
    history_path = os.path.join(model_dir, 'training_history.json')
    hyperparams_path = os.path.join(model_dir, 'hyperparameters.json')
    
    # 학습 내역 파일 존재 여부 확인
    if not os.path.exists(history_path):
        print(f"Error: '{history_path}' 파일이 존재하지 않습니다.")
        return
    
    # 학습 내역 로드
    with open(history_path, 'r') as f:
        history = json.load(f)
    
    # 하이퍼파라미터 로드 (선택 사항)
    hyperparameters = {}
    if os.path.exists(hyperparams_path):
        with open(hyperparams_path, 'r') as f:
            hyperparameters = json.load(f)
    
    # 손실 및 정확도 데이터 추출
    loss = history.get('loss')
    val_loss = history.get('val_loss')
    accuracy = history.get('accuracy')
    val_accuracy = history.get('val_accuracy')
    
    if not all([loss, val_loss, accuracy, val_accuracy]):
        print("Error: 학습 내역에 필요한 데이터가 부족합니다.")
        return
    
    epochs = range(1, len(loss) + 1)
    
    # 손실 그래프 시각화
    plt.figure(figsize=(14, 6))
    
    plt.subplot(1, 2, 1)
    plt.plot(epochs, loss, 'bo-', label='Train Loss')
    plt.plot(epochs, val_loss, 'ro-', label='Validation Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    
    # 정확도 그래프 시각화
    plt.subplot(1, 2, 2)
    plt.plot(epochs, accuracy, 'bo-', label='Train Accuracy')
    plt.plot(epochs, val_accuracy, 'ro-', label='Validation Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True)
    
    plt.tight_layout()
    plt.show()
    
    # 주요 학습 요약 정보 출력
    print("\n=== 학습 요약 정보 ===")
    best_val_acc = max(val_accuracy)
    best_epoch = val_accuracy.index(best_val_acc) + 1
    min_val_loss = min(val_loss)
    min_epoch = val_loss.index(min_val_loss) + 1
    
    print(f"최고 검증 정확도: {best_val_acc:.4f} (Epoch {best_epoch})")
    print(f"최소 검증 손실: {min_val_loss:.4f} (Epoch {min_epoch})")
    
    # 하이퍼파라미터 정보 출력 (선택 사항)
    if hyperparameters:
        print("\n=== 하이퍼파라미터 정보 ===")
        for key, value in hyperparameters.items():
            print(f"{key}: {value}")
    
    # 추가적인 그래프 저장 (선택 사항)
    # 저장하려면 아래 코드를 활성화하세요.
    
    plt.figure(figsize=(14, 6))
    
    plt.subplot(1, 2, 1)
    plt.plot(epochs, loss, label='Train Loss')
    plt.plot(epochs, val_loss, label='Validation Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    
    plt.subplot(1, 2, 2)
    plt.plot(epochs, accuracy, label='Train Accuracy')
    plt.plot(epochs, val_accuracy, label='Validation Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True)
    """
    plt.tight_layout()
    save_loss_graph_path = os.path.join(model_dir, 'loss_history.png')
    save_accuracy_graph_path = os.path.join(model_dir, 'accuracy_history.png')
    plt.savefig(save_loss_graph_path)
    plt.savefig(save_accuracy_graph_path)
    plt.close()
    
    print(f"\n손실 그래프 저장: {save_loss_graph_path}")
    print(f"정확도 그래프 저장: {save_accuracy_graph_path}")
    """


In [3]:
import os
import json
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt

def collect_and_evaluate_models(results_base_dir, test_data_dir, class_mode='binary', batch_size=32):
    """
    모델 디렉터리의 각 모델을 테스트 데이터로 평가하고, 결과를 Excel 파일로 저장합니다.

    :param results_base_dir: 모델 폴더들이 저장된 결과 디렉터리 경로
    :param test_data_dir: 테스트 데이터가 저장된 디렉터리 경로
    :param class_mode: 클래스 모드 ('binary' 또는 'categorical')
    :param batch_size: 배치 크기
    """
    model_performance_list = []

    # 결과를 저장할 Excel 파일 경로
    parent_dir = os.path.dirname(results_base_dir.rstrip('/\\'))
    output_excel_path = os.path.join(parent_dir, 'model_performance.xlsx')

    # 결과 디렉터리의 모든 하위 폴더 탐색
    for model_name in os.listdir(results_base_dir):
        model_dir = os.path.join(results_base_dir, model_name)
        if not os.path.isdir(model_dir):
            continue

        model_path = os.path.join(model_dir, 'model.h5')
        if not os.path.exists(model_path):
            print(f"모델 파일을 찾을 수 없습니다: {model_dir}")
            continue

        model_info = {'model_name': model_name}

        # 모델 로드
        try:
            model = load_model(model_path)
            print(f"모델 로드 완료: {model_name}")
        except Exception as e:
            print(f"모델 로드 중 오류 발생 ({model_name}): {e}")
            continue

        # 모델의 입력 크기 가져오기
        input_shape = model.input_shape[1:3]  # (height, width)
        print(f"모델의 입력 크기: {input_shape}")

        # 테스트 데이터 제너레이터 생성
        test_datagen = ImageDataGenerator(rescale=1./255)
        test_generator = test_datagen.flow_from_directory(
            test_data_dir,
            target_size=input_shape,
            batch_size=batch_size,
            class_mode=class_mode,
            shuffle=False
        )

        # 모델 평가
        try:
            loss, accuracy = model.evaluate(test_generator, verbose=0)
            print(f"모델: {model_name}, 테스트 손실: {loss:.4f}, 테스트 정확도: {accuracy:.4f}")
            model_info['test_accuracy'] = accuracy
            model_info['test_loss'] = loss
        except Exception as e:
            print(f"모델 평가 중 오류 발생 ({model_name}): {e}")
            model_info['test_accuracy'] = None
            model_info['test_loss'] = None

        # 훈련 기록 로드
        training_history_path = os.path.join(model_dir, 'training_history.json')
        if os.path.exists(training_history_path):
            try:
                with open(training_history_path, 'r') as f:
                    history = json.load(f)
                # 마지막 에포크의 훈련 및 검증 정확도와 손실 추출
                model_info['train_accuracy'] = history.get('accuracy', [None])[-1]
                model_info['train_loss'] = history.get('loss', [None])[-1]
                model_info['val_accuracy'] = history.get('val_accuracy', [None])[-1]
                model_info['val_loss'] = history.get('val_loss', [None])[-1]
            except Exception as e:
                print(f"{model_name}의 훈련 기록 로드 중 오류 발생: {e}")
                model_info['train_accuracy'] = None
                model_info['train_loss'] = None
                model_info['val_accuracy'] = None
                model_info['val_loss'] = None
        else:
            print(f"{model_name}의 훈련 기록을 찾을 수 없습니다.")
            model_info['train_accuracy'] = None
            model_info['train_loss'] = None
            model_info['val_accuracy'] = None
            model_info['val_loss'] = None

        # 모델 정보 리스트에 추가
        model_performance_list.append(model_info)

        # 메모리 정리
        del model

    # DataFrame 생성
    df = pd.DataFrame(model_performance_list)

    # DataFrame을 Excel 파일로 저장
    df.to_excel(output_excel_path, index=False)
    print(f"모델 성능 데이터가 {output_excel_path}에 저장되었습니다.")
